# Production Evaluation with MLflow

## Load Dependencies

In [0]:
%run ./step0_Environment

In [0]:
%run ./step0_Variables

# Environment Set up
Installing packages and configuring environment.

## Import model from UC registry

In [0]:
## Import model from UC registry
import mlflow

mlflow.set_registry_uri("databricks-uc")
model_uri = f"models:/{catalog_}.{schema_}.EmailClassifier_agent@champion"
mlflow.pyfunc.get_model_dependencies(model_uri)
EmailClassifier_agent = mlflow.pyfunc.load_model(model_uri)

2026/02/06 17:38:32 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /local_disk0/user_tmp_data/spark-170d1073-5768-4d0b-a0a2-5f/tmp0xv3sqiq/requirements.txt'.


2026/02/06 17:38:36 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - databricks-sdk (current: 0.85.0, required: databricks-sdk==0.82.0)
 - langchain-core (current: 1.2.9, required: langchain-core==1.2.7)
 - langgraph (current: 1.0.8, required: langgraph==1.0.7)
 - transformers (current: 5.1.0, required: transformers==5.0.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Unexpected internal error when monkey patching `PreTrainedModel.from_pretrained`: 
PreTrainedModel requires the PyTorch library but it was not found in your environment. Check out the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones

In [0]:
# Wrap model inside a callable function
def invoke_EmailClassifier_agent(input):
    return EmailClassifier_agent.predict({"input": input})

## Create evaluation dataset

In [0]:
eval_data_ = [
  {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "We received from: clara.rossi@example.com the following email: Hi, can I get a confirmation on my last request?",
                }
            ]
        },
        "expectations": {
          "expected_response": "Account & Data Requests",
        }
  },
  {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "We received from: marek.nowak@example.com the following email: I would like to know if I can update the delivery address for my order?",
                }
            ]
        },
        "expectations": {
          "expected_response": "Order Issues",
        }
  },
]

## Create custom judge
Create a variation of a built in judge [`mlflow.genai.scorers.Correctness()`](https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/concepts/judges/is_correct), to evaluate the emailemail label.

In [0]:
from mlflow.genai.scorers import scorer, Correctness

correctness_judge = Correctness()

@scorer
def classification_scorer(*, inputs, outputs, expectations, trace=None):
  
  ## Exctract label from output
  predicted_label = None
  if isinstance(outputs, dict):
      fin_output = outputs.get("final_output")
      if fin_output is not None:
          predicted_label = getattr(fin_output, "Label", None)
          if predicted_label is None and isinstance(fin_output, dict):
              predicted_label = fin_output.get("Label")
              
  # Delegate to built-in Correctness, overriding outputs
  return correctness_judge(
    inputs=inputs,
    outputs=predicted_label,
    expectations=expectations,
    trace = trace
  )

## Packages Install

# Global Variables definition
Creating global variables.

INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Error while sending or receiving.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 527, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
INFO:py4j.clientserver:Closing down clientserver connection
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 527, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
Python 3.12.3


INFO:py4j.clientserver:Error while sending or receiving.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 527, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
INFO:py4j.clientserver:Closing down clientserver connection
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 527, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/py

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.20.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.19.5, but you have protobuf 6.33.5 which is incompatible.
grpcio-status 1.67.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.5 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-health-checking 1.78.0 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.6 which is incompatible.
grpcio-tools 1.75.1 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.6 which is incompatible.
langgraph-a

## Configure Logging

## Run evaluation

In [0]:
eval_results = mlflow.genai.evaluate(
    data=eval_data_,
    predict_fn=invoke_EmailClassifier_agent,
    scorers=[classification_scorer]
)


2026/02/06 18:44:45 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.


Evaluating:   0%|          | 0/2 [Elapsed: 00:00, Remaining: ?] 